# FIT3182 Assignment Part B

- Name: Tan Chong Ern
- Student ID: 31435661
- Email: ctan0119@student.monash.edu
---

## Task 1. Processing Data Stream

### *a. Event Producer 1* 
*Write a python program that loads all the data from climate_streaming.csv and randomly (with replacement) feed the data to the stream every 10 seconds. You will need to append additional information such as producer information to identify the producer and created date.*

In [1]:
from kafka import KafkaProducer
from time import sleep
from json import dumps
import pandas as pd
import datetime as dt
import random
import pymongo


def publish(producer, topic, key, data):
    try:
        # encode the key and data into bytes
        byte_key = bytes(key, encoding = 'utf-8')
        byte_data = bytes(data, encoding = 'utf-8')
        producer.send(topic, key = byte_key, value = byte_data)
        producer.flush()
        print('Message sent successfully!')
    except Exception as e:
        print(f'Exception occured while publishing message: {str(e)}')

def connect_kafka_producer():
    producer = None
    try:
        producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'], api_version=(0, 10))
    except Exception as e:
        print(f'Exception occured while connecting to Kafka: {str(e)}')
    finally:
        return producer
    
def find_latest_date():
    # connect to the MongoDB client's historic collection
    client = pymongo.MongoClient()
    collection = client.fit_3182_assignment_db.historic

    # sort by the most recent date and return the top (latest) date
    res = collection.find().sort('date', -1).limit(1)
    latest_date = next(res, None)['date']
    return latest_date

def climate_csv_to_dict():
    # read the CSV file and convert it into a dictionary
    climate_df = pd.read_csv('climate_streaming.csv').to_dict('records')
    
    # separate the original precipitation values into a float and the precipitation flag/indicator
    for climate in climate_df:
        precipitation = climate.pop('precipitation ')
        climate['precipitation'] = float(precipitation[:-1])
        climate['precipitation_flag'] = precipitation[-1]

    return climate_df


if __name__ == '__main__':

    topic = 'climate_producer'
    climate_producer = connect_kafka_producer()
    latest_date = find_latest_date()
    climate_streaming = climate_csv_to_dict()

    print('Publishing records..')
    while True:
        latest_date += dt.timedelta(days = 1)
        randrow = climate_streaming[random.randrange(len(climate_streaming))]
        randrow['date'] = str(latest_date)
        publish(climate_producer, topic, 'climate_producer', dumps(randrow)) # inserts producer information into the key
        sleep(10)

Publishing records..
Message sent successfully!
Message sent successfully!
Message sent successfully!
Message sent successfully!
Message sent successfully!
Message sent successfully!
Message sent successfully!
Message sent successfully!
Message sent successfully!
Message sent successfully!
Message sent successfully!
Message sent successfully!
Message sent successfully!
Message sent successfully!


KeyboardInterrupt: 